In [1]:
# spark.stop()

In [2]:
import os
from pyspark.sql import SparkSession, types as t, functions as F

In [3]:
# https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-2.2.14.jar

spark = (
    SparkSession
    .builder
    # .master("spark://spark-master:7077")
    .appName("Testing Transformations")
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-2.2.14.jar") # GCS Connector
    .getOrCreate()
)

# Google Cloud Service Account Credentials
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile",os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))

spark

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/30 01:31:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
drug_schema = t.StructType(
    [
        t.StructField('patientid',t.StringType(),True),
        t.StructField('medicinalproduct',t.StringType(),True),
        t.StructField('activesubstancename',t.StringType(),True),
        t.StructField('drugadministrationroute',t.StringType(),True),
        t.StructField('drugstartdate',t.StringType(),True),
        t.StructField('drugenddate',t.StringType(),True),
        t.StructField('drugdosagetext',t.StringType(),True),
        t.StructField('drugstructuredosagenumb',t.StringType(),True),
        t.StructField('drugstructuredosageunit',t.StringType(),True),
        t.StructField('drugtreatmentduration',t.StringType(),True),
        t.StructField('drugtreatmentdurationunit',t.StringType(),True),
        t.StructField('drugrecurreadministration',t.StringType(),True),
    ]
)

reaction_schema = t.StructType(
    [
        t.StructField('patientid',t.StringType(),True),
        t.StructField('reactionmeddrapt',t.StringType(),True),
        t.StructField('reactionoutcome',t.StringType(),True),
    ]
)

In [5]:
bucket='gs://zoomcamp-454219-ade-pipeline/cleaned/pq/'

# Patient

### Load patient

In [6]:
# patient_schema = t.StructType(
#     [
#         t.StructField('patientid', t.StringType(),True),
#         t.StructField('patientagegroup', t.StringType(),True),
#         t.StructField('patientonsetage', t.StringType(),True),
#         t.StructField('patientonsetageunit', t.StringType(),True),
#         t.StructField('patientsex', t.StringType(),True),
#         t.StructField('patientweight', t.StringType(),True),
#         t.StructField('serious', t.IntegerType(), True),
#         t.StructField('seriousnessdeath', t.IntegerType(), True),
#         t.StructField('seriousnesshospitalization', t.IntegerType(), True),
#         t.StructField('seriousnessdisabling', t.IntegerType(), True),
#         t.StructField('seriousnesslifethreatening', t.IntegerType(), True),
#         t.StructField('seriousnessother', t.IntegerType(), True),
#         t.StructField('receivedate', t.IntegerType(), True),
#         t.StructField('receiptdate', t.IntegerType(), True),
#         t.StructField('safetyreportid', t.IntegerType(), True)
#     ]
# )

patient = (
    spark
    .read
    .parquet(bucket+'patient/*/*')
    )
print(f"Count: {patient.count()}")
print(patient.printSchema())
patient.show()

Count: 214812
root
 |-- patientid: string (nullable = true)
 |-- patientagegroup: string (nullable = true)
 |-- patientonsetage: integer (nullable = true)
 |-- patientonsetageunit: integer (nullable = true)
 |-- patientsex: integer (nullable = true)
 |-- patientweight: float (nullable = true)
 |-- serious: integer (nullable = true)
 |-- seriousnessdeath: integer (nullable = true)
 |-- seriousnesshospitalization: integer (nullable = true)
 |-- seriousnessdisabling: integer (nullable = true)
 |-- seriousnesslifethreatening: integer (nullable = true)
 |-- seriousnessother: integer (nullable = true)
 |-- receivedate: string (nullable = true)
 |-- receiptdate: string (nullable = true)
 |-- safetyreportid: string (nullable = true)

None


+--------------------+---------------+---------------+-------------------+----------+-------------+-------+----------------+--------------------------+--------------------+--------------------------+----------------+-----------+-----------+--------------+
|           patientid|patientagegroup|patientonsetage|patientonsetageunit|patientsex|patientweight|serious|seriousnessdeath|seriousnesshospitalization|seriousnessdisabling|seriousnesslifethreatening|seriousnessother|receivedate|receiptdate|safetyreportid|
+--------------------+---------------+---------------+-------------------+----------+-------------+-------+----------------+--------------------------+--------------------+--------------------------+----------------+-----------+-----------+--------------+
|0013f6bb-e446-4cd...|           null|           null|               null|      null|         null|   null|            null|                      null|                null|                      null|            null|       null|    

### Perform Transformation

In [7]:
patient.printSchema()

root
 |-- patientid: string (nullable = true)
 |-- patientagegroup: string (nullable = true)
 |-- patientonsetage: integer (nullable = true)
 |-- patientonsetageunit: integer (nullable = true)
 |-- patientsex: integer (nullable = true)
 |-- patientweight: float (nullable = true)
 |-- serious: integer (nullable = true)
 |-- seriousnessdeath: integer (nullable = true)
 |-- seriousnesshospitalization: integer (nullable = true)
 |-- seriousnessdisabling: integer (nullable = true)
 |-- seriousnesslifethreatening: integer (nullable = true)
 |-- seriousnessother: integer (nullable = true)
 |-- receivedate: string (nullable = true)
 |-- receiptdate: string (nullable = true)
 |-- safetyreportid: string (nullable = true)



In [8]:
# Patient Column transformations

patient = patient.withColumn(
    "patientagegroup",
    (
        F
        .when(F.col("patientagegroup") == '1', "Neonate")
        .when(F.col("patientagegroup") == '2', "Infant")
        .when(F.col("patientagegroup") == '3', "Child")
        .when(F.col("patientagegroup") == '4', "Adolescent")
        .when(F.col("patientagegroup") == '5', "Adult")
        .when(F.col("patientagegroup") == '6', "Elderly")
        .otherwise(None)
     )
)

# Normalize patientage
patient = patient.withColumn(
    "patientage",
    (
        F
        .when(F.col("patientonsetageunit") == 800, F.col("patientonsetage") * 10)
        .when(F.col("patientonsetageunit") == 801, F.col("patientonsetage") * 1)
        .when(F.col("patientonsetageunit") == 802, F.col("patientonsetage") / 12)
        .when(F.col("patientonsetageunit") == 803, F.col("patientonsetage") / 52.143)
        .when(F.col("patientonsetageunit") == 804, F.col("patientonsetage") / 365.25)
        .when(F.col("patientonsetageunit") == 805, F.col("patientonsetage") / 8766)
        .otherwise(None)
    ).cast(t.FloatType())
).drop("patientonsetageunit", "patientonsetage")

patient = patient.withColumn(
    "patientsex",
    (
        F
        .when(F.col("patientsex") == 1, "Male")
        .when(F.col("patientsex") == 2, "Female")
        .otherwise(None)
    ).cast(t.StringType())
)

patient = patient.withColumn(
    "patientweight",
    (
        F
        .when(
            F.col("patientweight").rlike(r"^\d+(\.\d+)?$"),
            F.col("patientweight").cast(t.FloatType()))
        .otherwise(None)
     )
)

patient = patient.withColumn(
    "serious",
    (
        F
        .when(F.col("serious") == 1, True)
        .when(F.col("serious") == 2, False)
        .otherwise(None)
     )
)

patient = patient.withColumn(
    "seriousnessdeath",
    (
        F
        .when(F.col("seriousnessdeath") == 1, True)
        .otherwise(False)
     )
)

patient = patient.withColumn(
    "seriousnesshospitalization",
    (
        F
        .when(F.col("seriousnesshospitalization") == 1, True)
        .otherwise(False)
     )
)

patient = patient.withColumn(
    "seriousnessdisabling",
    (
        F
        .when(F.col("seriousnessdisabling") == 1, True)
        .otherwise(False)
     )
)

patient = patient.withColumn(
    "seriousnesslifethreatening",
    (
        F
        .when(F.col("seriousnesslifethreatening") == 1, True)
        .otherwise(False)
     )
)

patient = patient.withColumn(
    "seriousnessother",
    (
        F
        .when(F.col("seriousnessother") == 1, True)
        .otherwise(False)
     )
)

In [9]:
patient.printSchema()

root
 |-- patientid: string (nullable = true)
 |-- patientagegroup: string (nullable = true)
 |-- patientsex: string (nullable = true)
 |-- patientweight: float (nullable = true)
 |-- serious: boolean (nullable = true)
 |-- seriousnessdeath: boolean (nullable = false)
 |-- seriousnesshospitalization: boolean (nullable = false)
 |-- seriousnessdisabling: boolean (nullable = false)
 |-- seriousnesslifethreatening: boolean (nullable = false)
 |-- seriousnessother: boolean (nullable = false)
 |-- receivedate: string (nullable = true)
 |-- receiptdate: string (nullable = true)
 |-- safetyreportid: string (nullable = true)
 |-- patientage: float (nullable = true)



### Display rows

In [10]:
patient.show()

+--------------------+---------------+----------+-------------+-------+----------------+--------------------------+--------------------+--------------------------+----------------+-----------+-----------+--------------+-----------+
|           patientid|patientagegroup|patientsex|patientweight|serious|seriousnessdeath|seriousnesshospitalization|seriousnessdisabling|seriousnesslifethreatening|seriousnessother|receivedate|receiptdate|safetyreportid| patientage|
+--------------------+---------------+----------+-------------+-------+----------------+--------------------------+--------------------+--------------------------+----------------+-----------+-----------+--------------+-----------+
|0013f6bb-e446-4cd...|           null|      null|         null|   null|           false|                     false|               false|                     false|           false|       null|       null|          null|       null|
|d8a5f44e-6e0c-4ac...|           null|    Female|          3.0|   null| 

In [11]:
patient.groupBy("patientagegroup").count().show()

+---------------+------+
|patientagegroup| count|
+---------------+------+
|           null|214765|
|          Adult|    28|
|        Elderly|    17|
|     Adolescent|     1|
|          Child|     1|
+---------------+------+



# Drug

### Load Drug

In [12]:
drug = (
    spark
    .read
    .parquet(bucket+'drug/*/*')
    )

drug.printSchema()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: gs://zoomcamp-454219-ade-pipeline/cleaned/pq/drug/*/*.

### Perform Cast

In [23]:
# Cast
drug = (
    drug
    .withColumn("drugstructuredosageunit", F.col("drugstructuredosageunit").cast(t.StringType()))
    .withColumn("drugrecurreadministration", F.col("drugrecurreadministration").cast(t.IntegerType()))
)

### Perform Transformation

In [24]:
route_mapping = {
    "001": "Auricular (otic)",
    "002": "Buccal",
    "003": "Cutaneous",
    "004": "Dental",
    "005": "Endocervical",
    "006": "Endosinusial",
    "007": "Endotracheal",
    "008": "Epidural",
    "009": "Extra-amniotic",
    "010": "Hemodialysis",
    "011": "Intra corpus cavernosum",
    "012": "Intra-amniotic",
    "013": "Intra-arterial",
    "014": "Intra-articular",
    "015": "Intra-uterine",
    "016": "Intracardiac",
    "017": "Intracavernous",
    "018": "Intracerebral",
    "019": "Intracervical",
    "020": "Intracisternal",
    "021": "Intracorneal",
    "022": "Intracoronary",
    "023": "Intradermal",
    "024": "Intradiscal (intraspinal)",
    "025": "Intrahepatic",
    "026": "Intralesional",
    "027": "Intralymphatic",
    "028": "Intramedullar (bone marrow)",
    "029": "Intrameningeal",
    "030": "Intramuscular",
    "031": "Intraocular",
    "032": "Intrapericardial",
    "033": "Intraperitoneal",
    "034": "Intrapleural",
    "035": "Intrasynovial",
    "036": "Intratumor",
    "037": "Intrathecal",
    "038": "Intrathoracic",
    "039": "Intratracheal",
    "040": "Intravenous bolus",
    "041": "Intravenous drip",
    "042": "Intravenous (not otherwise specified)",
    "043": "Intravesical",
    "044": "Iontophoresis",
    "045": "Nasal",
    "046": "Occlusive dressing technique",
    "047": "Ophthalmic",
    "048": "Oral",
    "049": "Oropharingeal",
    "050": "Other",
    "051": "Parenteral",
    "052": "Periarticular",
    "053": "Perineural",
    "054": "Rectal",
    "055": "Respiratory (inhalation)",
    "056": "Retrobulbar",
    "057": "Sunconjunctival",
    "058": "Subcutaneous",
    "059": "Subdermal",
    "060": "Sublingual",
    "061": "Topical",
    "062": "Transdermal",
    "063": "Transmammary",
    "064": "Transplacental",
    "065": "Unknown",
    "066": "Urethral",
    "067": "Vaginal"
}

# Fix missing parts of the date
drug = (
    drug
    .withColumn(
        "drugstartdate",
        (
            F
            .when(F.length("drugstartdate") == 4, F.concat("drugstartdate",F.lit("0101")))
            .when(F.length("drugstartdate") == 6, F.concat("drugstartdate",F.lit("01")))
            .otherwise(F.col("drugstartdate"))
        )
    )
)

drug = (
    drug
    .withColumn(
        "drugstartdate",
        (
            F.to_date("drugstartdate","yyyyMMdd")
        )
    )
)

drug = (
    drug
    .withColumn(
        "drugenddate",
        (
            F
            .when(F.length("drugenddate") == 4, F.concat("drugenddate",F.lit("0101")))
            .when(F.length("drugenddate") == 6, F.concat("drugenddate",F.lit("01")))
            .otherwise(F.col("drugenddate"))
        )
    )
)

drug = (
    drug
    .withColumn(
        "drugenddate",
        (
            F.to_date("drugenddate","yyyyMMdd")
        )
    )
)

map_expr = F.create_map([F.lit(i) for i in sum(route_mapping.items(),())])

drug = (
    drug
    .withColumn(
        "drugadministrationroute",
        map_expr[F.col("drugadministrationroute")]
    )
)

In [25]:
drug.show()

+--------------------+--------------------+-------------------+--------------------+-----------------------+-------------+-----------+--------------------+-----------------------+-----------------------+---------------------+-------------------------+-------------------------+
|           patientid|    medicinalproduct|activesubstancename|      drugindication|drugadministrationroute|drugstartdate|drugenddate|      drugdosagetext|drugstructuredosagenumb|drugstructuredosageunit|drugtreatmentduration|drugtreatmentdurationunit|drugrecurreadministration|
+--------------------+--------------------+-------------------+--------------------+-----------------------+-------------+-----------+--------------------+-----------------------+-----------------------+---------------------+-------------------------+-------------------------+
|36f3c70f-1044-423...|      ETHAMBUTOL HCL|               null|                null|                   null|         null|       null|                null|           

# Reaction

### Load Reaction

In [26]:
reaction = (
    spark
    .read
    .parquet(bucket+'reaction/*/*')
)

reaction.printSchema()

root
 |-- patientid: string (nullable = true)
 |-- reactionmeddrapt: string (nullable = true)
 |-- reactionoutcome: string (nullable = true)



### Perform Cast

In [27]:
# Cast
reaction = (
    reaction
    .withColumn("reactionoutcome", F.col("reactionoutcome").cast(t.IntegerType()))
)
reaction.printSchema()

root
 |-- patientid: string (nullable = true)
 |-- reactionmeddrapt: string (nullable = true)
 |-- reactionoutcome: integer (nullable = true)

